# The CUTHI code

In [15]:
def cuthi(x,y,stations, s=0.5, alpha=2):
    for i in range(len(stations)):
        if stations[i][0] == x and stations[i][1] == y:
            return stations[i][2]

    wtotal=0
    for k in range(len(stations)):
        r = ((stations[k,0]-x)**2+(stations[k,1]-y)**2)  # it's r^2
        if r!=0:
            w=1./r**(alpha/2)
            wcuthi = 1.
            for m in range(len(stations)):
                adist2 = float((stations[k][0] - stations[m][0]) ** 2 + (stations[k][1] - stations[m][1]) ** 2)
                bdist2 = float((stations[m][0] - x) ** 2 + (stations[m][1] - y) ** 2)
                if adist2>0 and bdist2>0:
                    cosgama = (adist2 + bdist2 - r) / (2.0 * adist2**.5 * bdist2**.5)
                    nangle = ((cosgama+1.)/2.)
                    if nangle<0:
                        if nangle<-0.01:
                            print ('nangle ngnalge')
                        else:
                            nangle = 0.
                            
                    wcuthi*=nangle**s
            w = w*wcuthi        
            wtotal+=w
    res=0
    for k in range(len(stations)):
        r = ((stations[k,0]-x)**2+(stations[k,1]-y)**2)
        if r == 0:
            res = stations[k,2]
        else:
            w=1./r**(alpha/2)
            wcuthi=1.
            for m in range(len(stations)):
                adist2 = float((stations[k][0] - stations[m][0]) ** 2 + (stations[k][1] - stations[m][1]) ** 2)
                bdist2 = float((stations[m][0] - x) ** 2 + (stations[m][1] - y) ** 2)
                if adist2>0 and bdist2>0:
                    cosgama = (adist2 + bdist2 - r) / (2.0 * adist2**.5 * bdist2**.5)
                    nangle = ((cosgama+1.)/2.)
                    if nangle<0:
                        if nangle<-0.01:
                            print ('nangle ngnalge')
                        else:
                            nangle = 0.
                    wcuthi*=nangle**s
            w = w*wcuthi
            res+=stations[k,2]*w/wtotal
    
    return res

# Cross-validation code

In [16]:
import sklearn.metrics
def crossvalidation(stations, alpha = 2):
    size=len(stations)
    r2 = 0
    resultcuthi = []
    for i in range(size):
            cv = []
            for j in range(size):
                if i!=j:
                   cv.append(stations[j])                      
            cv = np.asarray(cv)
            resultcuthi.append(cuthi(stations[i,0],stations[i,1],cv, s=0.5, alpha=alpha))
    r2   += sklearn.metrics.r2_score(stations[:,2], resultcuthi)
    return r2

# Build the stations data #

In [13]:
import random 
import numpy as np
def getFunction(size):
    minlat = -50
    maxlat = 150
    minlng = -50
    maxlng = 150
    
    stations = []
    for i in range (size):
        latrand = random.uniform(minlat, maxlat)
        lngrand = random.uniform(minlng, maxlng)
        z = latrand **2 - lngrand **2
        stations.append([lngrand, latrand, z])
    stations = np.asarray(stations)
    
    return stations

# Main code
1. build the station data (using mathematical function)
2. calculate the CUTHI accuracy using cross-validation (optional) - it is better to use interpolation only if the score is positive
3. interpolation

In [34]:
size=60
stations = getFunction(size)
print("Cross validation score is:",crossvalidation(stations), " should be positive")
resultcuthi = cuthi(130,130,stations)
print("Interpolated value of x=0, y=0 (should be 0, maximum value is 22500) is:", cuthi(0,0,stations))

Cross validation score is: 0.9704658470385893  should be positive
Interpolated value of x=0, y=0 (should be 0, maximum value is 22500) is: -15.327497137866354
